In [1]:
!pip install  --upgrade "transformers"   "datasets"  "accelerate"  "evaluate"  "bitsandbytes"  "trl"  "peft" 
!pip install tensorboardX
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Atte

In [2]:
!ngrok config add-authtoken 2fit1joWazA2g1HFUROudShO7KI_2siPG2oNzQDpo8RdgjVCm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [3]:
from flask import Flask, request, jsonify
from time import time
import torch
import transformers
from pyngrok import ngrok
import threading
import pandas as pd
import json
import numpy as np
import os
from huggingface_hub import notebook_login

from peft import LoraConfig, PeftConfig, PeftModel
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [4]:
app = Flask(__name__)
pipeline = None

In [5]:
temperature=0.7
max_length=512
top_p=0.9
model_name = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"

In [6]:
peft_model_id = "zaid178/llama3.2-1b-instruct-text-to-sql-order_values"
config = PeftConfig.from_pretrained(peft_model_id)

adapter_config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

In [7]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
#     quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model.load_adapter(peft_model_id)

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_id,
                                          padding_side = "right",
                                          add_eos_token = True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [10]:
try:
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer, 
        torch_dtype=torch.float32,
        device_map="auto",
)
except Exception as e:
    print(f"Failed to load the model: {e}") 
    raise RuntimeError("Failed to initialize the model. Please check the model path and configurations.")

In [11]:
def query_model(input_prompt,system_message):
    start_time = time()
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": input_prompt},
    ]

    try:
        prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
    except Exception as e:
        raise RuntimeError(f"Error applying chat template: {e}")
    
#     terminators = [
#         pipeline.tokenizer.eos_token_id,

#         pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
#     ]
    try:
        sequences = pipeline(
            prompt,
            do_sample=True,
            top_p=top_p,
            temperature=temperature,
#             eos_token_id=terminators,
            max_new_tokens=max_length,
            return_full_text=False,
#             pad_token_id=pipeline.model.config.eos_token_id
    )
    except Exception as e:
        raise RuntimeError(f"Error during model generation: {e}")
    
    generated_response = sequences[0]
    end_time = time()
    ttime = round(end_time - start_time, 2)

    return generated_response, ttime

In [12]:
@app.route('/v1/llama/response', methods=['POST'])
def query():
    if pipeline is None:
        return jsonify({"error": "Model not initialized properly"}), 500

    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "Invalid JSON format"}), 400
        
        input_prompt = data.get('input_prompt')
        system_message = data.get('system_message')
        

        if not input_prompt or not system_message:
            return jsonify({"error": "Missing 'input_prompt' and 'system_message'"}), 400
        
        generated_response, ttime = query_model(input_prompt,system_message)
        return jsonify({"generated_response":generated_response,
                       "response_time":ttime})
    
    except KeyError as e:
        return jsonify({"error": f"Missing required parameter: {str(e)}"}), 400
    except RuntimeError as e:
        return jsonify({"error": str(e)}), 500
    except Exception as e:
        return jsonify({"error": f"An unexpected error occurred: {str(e)}"}), 500

@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Not Found"}), 404

@app.errorhandler(500)
def internal_error(error):
    return jsonify({"error": "Internal Server Error"}), 500

def run_flask_app():
    app.run(host='0.0.0.0', port=3008)

if __name__ == '__main__':
    ngrok_tunnel = ngrok.connect(3008)
    print(f'Public URL: {ngrok_tunnel.public_url}')
    thread = threading.Thread(target=run_flask_app)
    thread.start()

Public URL: https://12a7-104-199-116-56.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
